In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import time

In [4]:
cam = cv2.VideoCapture(0)
#cam.set(cv2.CAP_PROP_AUTO_EXPOSURE, 0.25)    # Needed to set exposure manually
#cam.set(cv2.CAP_PROP_EXPOSURE, 900)          # 900ms exposure as per SOST
#cam.set(cv2.CAP_PROP_FPS, (1/0.9)) 

[ WARN:0] global ../modules/videoio/src/cap_gstreamer.cpp (501) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created


In [5]:
def take_image():
    for _ in range(10):
        r, _ = cam.read()
        if not r:
            break
    time.sleep(0.1)
    _, image = cam.read()
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [6]:
THRESHOLD = 50

def find_color(color_index, image):
    color_channel = image[:, :, color_index]
    channel_indices = [ i for i in range(3) if i != color_index ]
    other_channels = np.max(image[:, :, channel_indices], axis=-1).astype(np.uint16)
    diff = color_channel > (other_channels + THRESHOLD)
    diff = diff.astype(np.uint8)
    kernel = np.ones((5, 5), np.uint8) 
    diff = cv2.erode(diff, kernel, iterations=1) 
    diff = cv2.dilate(diff, kernel, iterations=1)
    return diff

In [7]:
def find_center_of_mass(diff):
    contours = cv2.findContours(diff, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    if len(contours) == 0:
        return None, None, None
    contours = sorted(contours, key=cv2.contourArea, reverse=True)
    contour = contours[0]
    
    moments = cv2.moments(contour)
    return int(moments['m10'] // moments['m00']), int(moments['m01'] // moments['m00']), contour

In [10]:
def find_marker(color_index):
    image = take_image()
    diff = find_color(color_index, image)
    x, y, contour = find_center_of_mass(diff)

    if (x is not None) and (y is not None) and (contour is not None):
        image = cv2.drawContours(image, [contour], -1, (0, 255, 255), 3)  # in green
        #plt.imshow(image)
        #plt.show()
        return x, y
    return None, None

In [12]:
#find_marker(0), find_marker(1), 
s = time.time()
print(find_marker(0))
print(time.time() - s)

(448, 187)
0.8568658828735352


In [48]:
from driver import Driver, normalize

d = Driver()
d.calibrate()

In [49]:
d.set_north()

Defining 308.52960186934945 as north


In [34]:
d.read_heading()

309.542510337703

In [64]:
STEP_SIZE = 10
BAND = 40
START_ANGLE = 80
ANGLES = [ i * STEP_SIZE - START_ANGLE for i in range(((START_ANGLE * 2) // STEP_SIZE) + 1) ]

MARKERS = [0, 2]

def get_marker_angle(color_index, angle):
    x, y = find_marker(color_index)
    if (x is not None) and (y is not None):
        if (x >= 320 - BAND) and (x <= 320 + BAND):
            return normalize(angle)
        elif x <= 320:
            return normalize(angle - STEP_SIZE / 2)
    return None

def find_markers(driver):
    detections = {}
    driver.turn_north(ANGLES[0])
    for angle in ANGLES:
        driver.turn_north(angle)
        for color_index in MARKERS:
            if len(set(MARKERS) - set(detections.keys())) == 0:
                return detections
            if color_index in detections:
                continue

            detection_angle = get_marker_angle(color_index, angle)
            if detection_angle is not None:
                print('Found', color_index, 'at angle', detection_angle)
                detections[color_index] = detection_angle
    return detections

def center_marker(color_index, driver):
    driver.turn_north(ANGLES[0])
    for angle in ANGLES:
        driver.turn_north(angle)
        detection_angle = get_marker_angle(color_index, angle)
        if detection_angle is not None:
            return detection_angle
    return None

In [ ]:
find_markers(d)

In [60]:
center_marker(0, d)

335.0

In [61]:
center_marker(2, d)

20

In [3]:
cam.release()
del cam